In [2]:
import ast
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import os
os.chdir('../')
import pandas as pd

from pathlib import Path

from src.utils.utils import GetSortedSpeciesCode
from src.utils.threshold_app import countFileLabels, findThreshold

In [3]:
TARGET_SPECIES = GetSortedSpeciesCode()

In [5]:
df = pd.read_csv(Path.cwd().joinpath('data', 'test_app.csv'), header=0)

/var/folders/hh/7m7phfcn0ygb6lqmkknry15r0000gn/T/ipykernel_28353/3448619655.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Path.cwd().joinpath('data', 'test_app.csv'), header=0)


,Unnamed: 0,AA,AA.1,AA.2,AA.3,AA.4,CR,CR.1,CR.2,CR.3,...,end time,AA(P),CR(P),FH(P),HA(P),NV(P),PAL(P),PM(P),SE(P),YB(P)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.00,0.004185,0.248091,0.057135,0.002898,0.004821,0.280541,0.245639,0.004287,0.009122
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.25,0.000149,0.062220,0.003618,0.000147,0.005521,0.901762,0.040678,0.000618,0.000163
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.50,0.012076,0.015404,0.016387,0.000180,0.002398,0.943791,0.011357,0.000737,0.001514
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.75,0.940270,0.018211,0.019348,0.000254,0.002177,0.143162,0.026854,0.000794,0.005348
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.00,0.858117,0.008553,0.002660,0.000101,0.072063,0.055498,0.067302,0.000677,0.000508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.25,0.008842,0.002447,0.003505,0.248103,0.035464,0.005426,0.233551,0.001578,0.300304
393102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.50,0.011527,0.003761,0.000796,0.020646,0.085399,0.001779,0.178037,0.001673,0.724665
393103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,69.75,0.001444,0.010678,0.000153,0.007639,0.677295,0.021077,0.128220,0.000066,0.000305
393104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70.00,0.115158,0.076902,0.000991,0.006559,0.062100,0.014212,0.093877,0.000570,0.005560


In [50]:
df1 = pd.read_csv(Path.cwd().joinpath('data', 'daily-record.csv'), header=0)
df1['file'] = df1['file'].apply(lambda x: str(x).split('\\')[1])
df2 = pd.read_csv(Path.cwd().joinpath('data', 'gw-count.csv'), header=0)
df2['file'] = df2['file'].apply(lambda x: str(x).split('/')[1])
df3 = pd.merge(df1, df2, on='file', suffixes=['(T)', ''])
df3.to_csv(Path.cwd().joinpath('data', 'test-compare.csv'), header=True, index=False)

In [12]:
df = pd.read_csv(Path.cwd().joinpath('data', 'label_auto.csv'), header=0)
df = df[['file']+TARGET_SPECIES]
df = df.groupby(['file']).sum()
# df.to_csv(Path.cwd().joinpath('data', 'daily-record.csv'), header=True, index=True)
df.shape

(4806, 9)

In [11]:
df = pd.read_csv(Path.cwd().joinpath('data', 'test_overall.csv'), header=[0, 1], index_col=0)
dfIndex = df.index
thres = []
for sp in TARGET_SPECIES:
  thres.append(dfIndex[np.argmax(df.loc[:, (sp, 'f0.5')])])

thres

[0.55, 0.71, 0.68, 0.54, 0.62, 0.66, 0.55, 0.44, 0.59]

In [ ]:
sLabelPaths = sorted(Path.cwd().joinpath('data', 'raw', 'Label').glob('*.txt'))
countDF = countFileLabels(sLabelPaths)

## Load Predict Probability CSV
predictDF = pd.read_csv(Path.cwd().joinpath('data', 'TEST_APP.csv'), header=0)

## Find threshold for each species
maxThresDict, spThresDict = findThreshold(predictDF, countDF)

In [ ]:
df = pd.DataFrame.from_dict(spThresDict)
precisionRecallDF = df.applymap(lambda x: x[2:4])
precisionRecallDF['threshold'] = np.around(np.arange(0, 1, 0.05), decimals=2)
precisionRecallDF=precisionRecallDF[['threshold']+TARGET_SPECIES]
precisionRecallDF[['threshold']+TARGET_SPECIES].to_csv(Path.cwd().joinpath('precisionRecall.csv'), header=True, index=False)

In [ ]:
f = lambda x,y,z : (1+z**2) * (x * y) / (z**2 * x + y)

resDFIndex = pd.MultiIndex.from_product(
  [TARGET_SPECIES, ['precision', 'recall', 'f0.5', 'f1', 'f2']], 
)
thresList = np.around(np.arange(0, 1, 0.05), decimals=2)
resDF = pd.DataFrame(columns=thresList, index=resDFIndex)

for sp in TARGET_SPECIES:
  for index, row in enumerate(precisionRecallDF[sp]):
    resDF.loc[(sp, 'precision'), thresList[index]] = np.round(row[0], decimals=4)
    resDF.loc[(sp, 'recall'), thresList[index]] = np.round(row[1], decimals=4)
    resDF.loc[(sp, 'f0.5'), thresList[index]] = np.round(f(row[0], row[1], 0.5), decimals=4)
    resDF.loc[(sp, 'f1'), thresList[index]] = np.round(f(row[0], row[1], 1.0), decimals=4)
    resDF.loc[(sp, 'f2'), thresList[index]] = np.round(f(row[0], row[1], 2.0), decimals=4)

resDF.fillna(0, inplace=True)
resDF.T.to_csv(Path.cwd().joinpath('one-min-report.csv'), header=True, index=True)

In [ ]:
def filterTarget(l):
  res = []
  for x in l:
    if x in TARGET_SPECIES:
      res.append(x)
  return res

def countLabel(df:pd.DataFrame):
  spDict = {sp:0 for sp in TARGET_SPECIES}
  for _, row in df.iterrows():
    for sp in row['label']:
      if sp in TARGET_SPECIES:
        spDict[sp] += 1
  return spDict

In [ ]:
df = pd.read_csv(Path.cwd().joinpath('data', 'LABEL_SEG.csv'), header=0)
df['label'] = df['label'].apply(lambda x: filterTarget(ast.literal_eval(x)))

xcDF = df[df['file'].str.contains('XC\d')]
mlDF = df[df['file'].str.contains('ML\d')]
selfDF = df[~df['file'].str.contains('ML\d|XC\d')]

xcLabel = countLabel(xcDF)
mlLabel = countLabel(mlDF)
selfLabel = countLabel(selfDF)

In [ ]:
labelDF = pd.DataFrame.from_records(
  [xcLabel, mlLabel, selfLabel],
  index=['XC', 'ML', 'SELF']
)
labelDF = labelDF.T

plt.style.use('ggplot')
fig = plt.figure(figsize=(16, 9))
ax = plt.gca()
ax.bar(x=TARGET_SPECIES, height=labelDF['XC']+labelDF['ML']+labelDF['SELF'])
ax.bar(x=TARGET_SPECIES, height=labelDF['XC']+labelDF['SELF'])
ax.bar(x=TARGET_SPECIES, height=labelDF['SELF'])
print(labelDF)

In [ ]:
import librosa
import librosa.display

audio, sr = librosa.load(
  str(Path.cwd().joinpath('data', 'raw', 'GW01FOREST_20210401_060600.wav')),
  sr=None
)
nrAudio, sr = librosa.load(
  str(Path.cwd().joinpath('data', 'raw', 'NrAudio', 'GW01FOREST_20210401_060600.wav')),
  sr=None
)
mel = librosa.feature.melspectrogram(
  y=audio, sr=sr, n_fft=1024, hop_length=512, 
  n_mels=128, fmin=1000, fmax=10000
)
nrMel = librosa.feature.melspectrogram(
  y=nrAudio, sr=sr, n_fft=1024, hop_length=512, 
  n_mels=128, fmin=1000, fmax=10000
)
image = librosa.pcen(mel * (2**31), sr=sr, time_constant=0.06, gain=0.8, bias=10, power=0.25)
nrImage = librosa.pcen(nrMel * (2**31), sr=sr, time_constant=0.06, gain=0.8, bias=10, power=0.25)


fig, ax = plt.subplots(4, 1, figsize=(128, 72))
ax = ax.flatten()
librosa.display.waveshow(audio, sr=sr, ax=ax[0], color='b')
ax[0].set_xlim(0, 60)
librosa.display.specshow(
  image, y_axis='linear', x_axis='time',
  sr=sr, ax=ax[1], fmin=1000, fmax=10000, cmap='YlOrRd'
)
librosa.display.waveshow(nrAudio, sr=sr, ax=ax[2], color='b')
ax[2].set_xlim(0, 60)
librosa.display.specshow(
  nrImage, y_axis='linear', x_axis='time',
  sr=sr, ax=ax[3], fmin=1000, fmax=10000, cmap='YlOrRd'
)